In [3]:
#데이터 읽어오기
import pandas as pd
MPI_df = pd.read_excel('./data/채굴자 포지션 지표 (MPI).xlsx')
df_interval = pd.read_excel('./data/변동성_구간.xlsx')

# 데이터전처리
MPI_df['날짜'] = pd.to_datetime(MPI_df['날짜'], format = "%Y %m %d")
MPI_df['가격'] = MPI_df['가격'].str.replace(',', '').astype(float)
MPI_df['가격'] = pd.to_numeric(MPI_df['가격'], errors='coerce')
new_columns = {'날짜':'DATE','가격':'PRICE','채굴자 포지션 지표 (MPI)':'MPI'}
MPI_df = MPI_df.rename(columns=new_columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091 entries, 0 to 1090
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   날짜                1091 non-null   datetime64[ns]
 1   가격                1091 non-null   float64       
 2   채굴자 포지션 지표 (MPI)  1091 non-null   float64       
 3   label             1091 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 34.2 KB


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN

#경고무시
import warnings
warnings.filterwarnings("ignore")

sns.set(style="whitegrid", palette="pastel", color_codes=True)
sns.set_context('talk')

#스케일링(정구화)
scaler = MinMaxScaler()
MPI_df[['scaled_MPI']] = scaler.fit_transform(MPI_df[['MPI']])
X = MPI_df[['scaled_MPI']].values

#파라미터값생성
eps = [i*(1/100) for i in range(1,10)]


In [ ]:
from datetime import datetime, timedelta
fig, axs = plt.subplots(3, 3, figsize=(30, 21))


for i in range(3):
    for j in range(3):
        ax1 = axs[i][j]

        # DBSCAN 모델 생성 및 학습
        model = DBSCAN(eps=eps[i * 3 + j], min_samples=20)
        model.fit(X)
        labels = model.labels_
        MPI_df['label'] = labels
        # 첫 번째 y축 그래프 (가격)
        ax1.plot(MPI_df['DATE'], MPI_df['PRICE'], label='price',color='#5b5b5b')
        # ax1.set_xlabel('DATE')
        # ax1.set_ylabel('PRICE',color='#5b5b5b')
        ax1.tick_params(axis='y',color='#5b5b5b')
        # ax1.legend(loc='upper left')
        ax1.tick_params(axis='x', labelrotation=45, labelsize=8)
        ax1.tick_params(axis='y', labelsize=8)
        # 두 번째 y축 그래프 (CDD)
        ax2 = ax1.twinx()
        ax2.plot(MPI_df['DATE'], MPI_df['scaled_MPI'], label='CDD',color='#912df1')
        # ax2.set_ylabel('CDD',color='#912df1')
        ax2.tick_params(axis='y',color='#912df1')

        # 이상치 표시
        label_minus_1 = MPI_df[MPI_df['label'] == -1]
        ax2.scatter(label_minus_1['DATE'], label_minus_1['scaled_MPI'], marker='x', color='black', label='CDD (-1)')

        # 구간 표시
        for index, row in df_interval.iterrows():
            start_date = row['start_date']
            end_date = row['end_date']
            ax1.axvspan(start_date, end_date, color='gray', alpha=0.5)

        # 그래프의 제목에 파라미터 값 추가
        ax1.set_title(f'eps = {eps[i * 3 + j]}, count:{len(label_minus_1)}')
        plt.grid(False)
fig.tight_layout()
plt.show()